In [1]:
## calculating GEDI summary stats from extracted dataframes

In [2]:
install.packages("s3")
#install.packages("doParallel")
#install.packages("RItools")    

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
library("terra")
library("dplyr")
library("sf")
library("s3")
library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#library("doParallel")
#library("RItools")

terra 1.7.29


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE



In [4]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
gediwk <- 24

In [8]:
iso3 <- "BaE"
gedi_paf <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf)

pa_metrics1 <- readRDS(gedi_paf[1]) %>% unique()
BaE_pa_ALL <- pa_metrics1
rm(pa_metrics1)
saveRDS(BaE_pa_ALL, file=paste(f.path,"WDPA_extract/",iso3,"_ALLpa_stats.RDS",sep=""))

for(i in 2:length(gedi_paf)){

    BaE_pa_ALL <- readRDS(paste(f.path,"WDPA_extract/",iso3,"_ALLpa_stats.RDS",sep=""))
    print(gedi_paf[i])
    pa_metrics <- readRDS(gedi_paf[i]) %>% unique()

    if (length(table(pa_metrics$status))<2) {
    cat(iso3, gedi_paf[i], "has 0 protected or treatment \n")
    } else if (table(pa_metrics$status)[1]!=0 && table(pa_metrics$status)[2]!=0) {
        BaE_pa_ALL <- rbind(BaE_pa_ALL, pa_metrics)  #dplyr::bind_rows(results_list)
    }
  print(nrow(BaE_pa_ALL))
  rm(pa_metrics)
    
  saveRDS(BaE_pa_ALL, file=paste(f.path,"WDPA_extract/",iso3,"_ALLpa_stats.RDS",sep="")) 
}

[1] 240

[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_10795_gedi_wk_24_conti_biome_TSMBF.RDS"
[1] 1376957
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_10801_gedi_wk_24_conti_biome_TSGSS+TSMBF.RDS"
[1] 2377655
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_10802_gedi_wk_24_conti_biome_TSMBF.RDS"
[1] 2982839
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_10803_gedi_wk_24_conti_biome_TSMBF.RDS"
[1] 3436722
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_115686_gedi_wk_24_conti_biome_TSMBF.RDS"
[1] 3923078
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_115700_gedi_wk_24_conti_biome_TSDBF.RDS"
[1] 3936982
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_115765_gedi_wk_24_conti_biome_TSGSS.RDS"
[1] 3964946
[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/Ba

ERROR: Error in saveRDS(BaE_pa_ALL, file = paste(f.path, "WDPA_extract/", iso3, : error writing to connection
